In [1]:
# import bibliotek
import os
import librosa
import pickle
import sklearn
import numpy

In [10]:
# 1. Ekstrakcja cech z próbek
# Lista ekstraktorów cech: https://librosa.org/doc/latest/feature.html

# ścieżka z folderami z nagraniami - próbki:
train_audio_path = '../data/'
commands = os.listdir(train_audio_path);  


for N in range(1,101):
    # użyjemy MFCC, tj. Mel-frequency cepstral coefficients
    # jednym z parametrów jest liczba wyjściowych współczynników:
    n_mfcc = N

    # wczytujemy każde nagranie, wykonujemy ekstrakcję i zapamiętujemy jej wynik
    classes = []
    class_size = 120 # użyjemy tylko po kilka danych
    samples = []
    labels = []
    print("\nClass: ", end = '')
    for command in commands:
        classes.append(command)
        print(command, end = ', ')
        sample_num = 0
        for file in os.listdir(train_audio_path + '/' + command + '/'):
            # print(".", end = '')
            sample_num += 1
            # przykład z MFCC:
            # y - waveform, sr - sampling rate, n_mfcc - number of MFCCs to return
            y, sr = librosa.load(train_audio_path + '/' + command + '/' + file)
            mfcc_seq = librosa.feature.mfcc(y = y, sr = sr, n_mfcc = n_mfcc)
            mean_mfcc_seq = []
            for feature in mfcc_seq:
                mean_mfcc_seq.append(numpy.mean(feature))
            samples.append(mean_mfcc_seq)
            labels.append(command)
            if sample_num == class_size:
                break

    # warto podglądnąć, dla mniejszych n_mfcc, np. 5 co kryje mfcc_seq i mean_mfcc_seq

    # zapiszmy bazę do późniejszej pracy
    database = open(f'database{N}.pkl', 'wb')
    pickle.dump([classes, samples, labels], database)
    database.close()
        


Class: bed, bird, cat, dog, down, eight, five, four, go, happy, house, left, marvin, nine, no, off, on, one, right, seven, sheila, six, stop, three, tree, two, up, wow, yes, zero, _background_noise_, 
Class: bed, bird, cat, dog, down, eight, five, four, go, happy, house, left, marvin, nine, no, off, on, one, right, seven, sheila, six, stop, three, tree, two, up, wow, yes, zero, _background_noise_, 
Class: bed, bird, cat, dog, down, eight, five, four, go, happy, house, left, marvin, nine, no, off, on, one, right, seven, sheila, six, stop, three, tree, two, up, wow, yes, zero, _background_noise_, 
Class: bed, bird, cat, dog, down, eight, five, four, go, happy, house, left, marvin, nine, no, off, on, one, right, seven, sheila, six, stop, three, tree, two, up, wow, yes, zero, _background_noise_, 
Class: bed, bird, cat, dog, down, eight, five, four, go, happy, house, left, marvin, nine, no, off, on, one, right, seven, sheila, six, stop, three, tree, two, up, wow, yes, zero, _background_noi

In [12]:
# 2. Wybór bez mfcc

# Zautomatyzowany i losowy sposób to użycie:
# "from sklearn.model_selection import train_test_split" oraz funkcji "train_test_split()"

best_mfcc = 0
best_quality = 0
for N in range(1,101):
    # wczytanie danych
    database = open(f'classes/database{N}.pkl', 'rb')
    classes, samples, labels = pickle.load(database)
    database.close()

    # wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
    # oraz kolejne 2 z każdej klasy jako testowe
    # uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
    train_size = 40
    test_size = 10
    train_samples = []
    train_labels = []
    test_samples = []
    test_labels = []
    for classname in classes:
        train_size_index = 0
        test_size_index = 0
        for i in range(len(labels)):
            if labels[i] == classname and train_size_index < train_size:
                train_samples.append(samples[i])
                train_labels.append(labels[i])
                train_size_index += 1
            elif labels[i] == classname and test_size_index < test_size:
                test_samples.append(samples[i])
                test_labels.append(labels[i])
                test_size_index += 1
            if train_size_index == train_size and test_size_index == test_size:
                break

    # proszę sprawdzić, czy dane zostały wybrane prawidłowo


    # 3. Uczenie klasyfikatora danymi treningowymi

    # Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
    # Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

    # skalowanie/normalizacja danych
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(train_samples)
    train_samples = scaler.transform(train_samples)
    test_samples = scaler.transform(test_samples)

    # uczenie
    from sklearn.neighbors import KNeighborsClassifier
    k = 5
    classifier = KNeighborsClassifier(n_neighbors = k)
    classifier.fit(train_samples, train_labels)


    # 4. Klasyfikacja nieznanych próbek
    predicted_labels = classifier.predict(test_samples)

    # zobaczmy wynik - zakomentować dla dużych zbiorów
    # print(test_labels)
    # print(predicted_labels)


    # 5. Ocena klasyfikatora
    # print(test_labels == predicted_labels)
    quality = numpy.sum(test_labels == predicted_labels) / len(test_labels) * 100;
    print(f'mfcc={N} Ocena klasyfikatora: %.2f %%' % quality)
    if (quality > best_quality):
        best_mfcc = N
        best_quality = quality

print(f"best_mfcc = {best_mfcc}; best_quality = {best_quality}")
n_mfcc_optimal = best_mfcc

# Optymalny n_mfcc = 9; quality = 11.67%

mfcc=1 Ocena klasyfikatora: 2.67 %
mfcc=2 Ocena klasyfikatora: 4.67 %
mfcc=3 Ocena klasyfikatora: 4.00 %
mfcc=4 Ocena klasyfikatora: 5.67 %
mfcc=5 Ocena klasyfikatora: 6.67 %
mfcc=6 Ocena klasyfikatora: 6.00 %
mfcc=7 Ocena klasyfikatora: 9.33 %
mfcc=8 Ocena klasyfikatora: 8.67 %
mfcc=9 Ocena klasyfikatora: 11.67 %
mfcc=10 Ocena klasyfikatora: 8.67 %
mfcc=11 Ocena klasyfikatora: 8.00 %
mfcc=12 Ocena klasyfikatora: 8.67 %
mfcc=13 Ocena klasyfikatora: 7.33 %
mfcc=14 Ocena klasyfikatora: 6.33 %
mfcc=15 Ocena klasyfikatora: 6.33 %
mfcc=16 Ocena klasyfikatora: 8.67 %
mfcc=17 Ocena klasyfikatora: 8.67 %
mfcc=18 Ocena klasyfikatora: 9.33 %
mfcc=19 Ocena klasyfikatora: 9.33 %
mfcc=20 Ocena klasyfikatora: 10.33 %
mfcc=21 Ocena klasyfikatora: 8.67 %
mfcc=22 Ocena klasyfikatora: 8.00 %
mfcc=23 Ocena klasyfikatora: 8.33 %
mfcc=24 Ocena klasyfikatora: 6.67 %
mfcc=25 Ocena klasyfikatora: 6.33 %
mfcc=26 Ocena klasyfikatora: 5.67 %
mfcc=27 Ocena klasyfikatora: 6.00 %
mfcc=28 Ocena klasyfikatora: 5.33 %

In [13]:
# 2. Dobór optymalny zbioru testowego

# Zautomatyzowany i losowy sposób to użycie:
# "from sklearn.model_selection import train_test_split" oraz funkcji "train_test_split()"

best_mfcc = n_mfcc_optimal

best_train_size = 0
best_quality = 0
for N in range(1,91):
    # wczytanie danych
    database = open(f'classes/database{best_mfcc}.pkl', 'rb')
    classes, samples, labels = pickle.load(database)
    database.close()

    # wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
    # oraz kolejne 2 z każdej klasy jako testowe
    # uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
    train_size = N
    test_size = 10
    train_samples = []
    train_labels = []
    test_samples = []
    test_labels = []
    for classname in classes:
        train_size_index = 0
        test_size_index = 0
        for i in range(len(labels)):
            if labels[i] == classname and train_size_index < train_size:
                train_samples.append(samples[i])
                train_labels.append(labels[i])
                train_size_index += 1
            elif labels[i] == classname and test_size_index < test_size:
                test_samples.append(samples[i])
                test_labels.append(labels[i])
                test_size_index += 1
            if train_size_index == train_size and test_size_index == test_size:
                break

    # proszę sprawdzić, czy dane zostały wybrane prawidłowo


    # 3. Uczenie klasyfikatora danymi treningowymi

    # Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
    # Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

    # skalowanie/normalizacja danych
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(train_samples)
    train_samples = scaler.transform(train_samples)
    test_samples = scaler.transform(test_samples)

    # uczenie
    from sklearn.neighbors import KNeighborsClassifier
    k = 5
    classifier = KNeighborsClassifier(n_neighbors = k)
    classifier.fit(train_samples, train_labels)


    # 4. Klasyfikacja nieznanych próbek
    predicted_labels = classifier.predict(test_samples)

    # zobaczmy wynik - zakomentować dla dużych zbiorów
    # print(test_labels)
    # print(predicted_labels)


    # 5. Ocena klasyfikatora
    # print(test_labels == predicted_labels)
    quality = numpy.sum(test_labels == predicted_labels) / len(test_labels) * 100;
    print(f'train_size={N} Ocena klasyfikatora: %.2f %%' % quality)
    if (quality > best_quality):
        best_train_size = N
        best_quality = quality

print(f"best_train_size = {best_train_size}; best_quality = {best_quality}")
train_size_optimal = best_train_size

# best_train_size = 12; best_quality = 15.666666666666668

train_size=1 Ocena klasyfikatora: 3.61 %
train_size=2 Ocena klasyfikatora: 5.59 %
train_size=3 Ocena klasyfikatora: 5.61 %
train_size=4 Ocena klasyfikatora: 7.28 %
train_size=5 Ocena klasyfikatora: 7.31 %
train_size=6 Ocena klasyfikatora: 8.33 %
train_size=7 Ocena klasyfikatora: 9.33 %
train_size=8 Ocena klasyfikatora: 11.33 %
train_size=9 Ocena klasyfikatora: 8.00 %
train_size=10 Ocena klasyfikatora: 10.33 %
train_size=11 Ocena klasyfikatora: 14.00 %
train_size=12 Ocena klasyfikatora: 15.67 %
train_size=13 Ocena klasyfikatora: 14.67 %
train_size=14 Ocena klasyfikatora: 11.67 %
train_size=15 Ocena klasyfikatora: 11.00 %
train_size=16 Ocena klasyfikatora: 10.00 %
train_size=17 Ocena klasyfikatora: 8.00 %
train_size=18 Ocena klasyfikatora: 6.00 %
train_size=19 Ocena klasyfikatora: 5.33 %
train_size=20 Ocena klasyfikatora: 7.33 %
train_size=21 Ocena klasyfikatora: 7.67 %
train_size=22 Ocena klasyfikatora: 7.00 %
train_size=23 Ocena klasyfikatora: 7.67 %
train_size=24 Ocena klasyfikatora: 

In [15]:
# 2. Dobór optymalny parametru K

# Zautomatyzowany i losowy sposób to użycie:
# "from sklearn.model_selection import train_test_split" oraz funkcji "train_test_split()"

best_mfcc = n_mfcc_optimal
best_train_size = train_size_optimal

best_K = 0
best_quality = 0
for N in range(1,11):
    # wczytanie danych
    database = open(f'classes/database{best_mfcc}.pkl', 'rb')
    classes, samples, labels = pickle.load(database)
    database.close()

    # wybierzmy 10 pierwszych próbek z każdej klasy jako uczące
    # oraz kolejne 2 z każdej klasy jako testowe
    # uwaga! train_size + test_size NIE może przekroczyć class_size (poprzedni punkt)
    train_size = best_train_size
    test_size = 10
    train_samples = []
    train_labels = []
    test_samples = []
    test_labels = []
    for classname in classes:
        train_size_index = 0
        test_size_index = 0
        for i in range(len(labels)):
            if labels[i] == classname and train_size_index < train_size:
                train_samples.append(samples[i])
                train_labels.append(labels[i])
                train_size_index += 1
            elif labels[i] == classname and test_size_index < test_size:
                test_samples.append(samples[i])
                test_labels.append(labels[i])
                test_size_index += 1
            if train_size_index == train_size and test_size_index == test_size:
                break

    # proszę sprawdzić, czy dane zostały wybrane prawidłowo


    # 3. Uczenie klasyfikatora danymi treningowymi

    # Lista klasyfikatorów: https://stackabuse.com/overview-of-classification-methods-in-python-with-scikit-learn/
    # Prosty tutorial: https://www.digitalocean.com/community/tutorials/how-to-build-a-machine-learning-classifier-in-python-with-scikit-learn

    # skalowanie/normalizacja danych
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(train_samples)
    train_samples = scaler.transform(train_samples)
    test_samples = scaler.transform(test_samples)

    # uczenie
    from sklearn.neighbors import KNeighborsClassifier
    k = N
    classifier = KNeighborsClassifier(n_neighbors = k)
    classifier.fit(train_samples, train_labels)


    # 4. Klasyfikacja nieznanych próbek
    predicted_labels = classifier.predict(test_samples)

    # zobaczmy wynik - zakomentować dla dużych zbiorów
    # print(test_labels)
    # print(predicted_labels)


    # 5. Ocena klasyfikatora
    # print(test_labels == predicted_labels)
    quality = numpy.sum(test_labels == predicted_labels) / len(test_labels) * 100;
    print(f'K={N} Ocena klasyfikatora: %.2f %%' % quality)
    if (quality > best_quality):
        best_K = N
        best_quality = quality

print(f"best_K = {best_K}; best_quality = {best_quality}")
k_optimal = best_K

# best_K = 3; best_quality = 17.0

K=1 Ocena klasyfikatora: 16.67 %
K=2 Ocena klasyfikatora: 15.33 %
K=3 Ocena klasyfikatora: 17.00 %
K=4 Ocena klasyfikatora: 16.67 %
K=5 Ocena klasyfikatora: 15.67 %
K=6 Ocena klasyfikatora: 15.67 %
K=7 Ocena klasyfikatora: 14.33 %
K=8 Ocena klasyfikatora: 15.33 %
K=9 Ocena klasyfikatora: 14.00 %
K=10 Ocena klasyfikatora: 12.00 %
best_K = 3; best_quality = 17.0


['dog', 'dog', 'cat', 'cat', 'zero', 'zero', 'on', 'on', 'go', 'go', 'eight', 'eight', 'nine', 'nine', 'house', 'house', 'happy', 'happy', 'up', 'up', 'off', 'off', 'five', 'five', 'four', 'four', 'no', 'no', 'six', 'six', 'seven', 'seven', 'one', 'one', 'yes', 'yes', 'wow', 'wow', 'marvin', 'marvin', 'stop', 'stop', 'right', 'right', 'down', 'down', 'bed', 'bed', 'left', 'left', 'two', 'two', 'tree', 'tree', 'bird', 'bird', 'three', 'three', 'sheila', 'sheila']
['down' 'off' 'cat' 'down' 'seven' 'left' 'happy' 'marvin' 'four' 'bed'
 'four' 'cat' 'bird' 'marvin' 'dog' 'four' 'cat' 'cat' 'bird' 'one' 'four'
 'eight' 'five' 'bed' 'house' 'bed' 'bird' 'down' 'cat' 'left' 'bed'
 'down' 'down' 'four' 'zero' 'cat' 'down' 'happy' 'bird' 'dog' 'down'
 'cat' 'bird' 'bird' 'down' 'cat' 'bed' 'sheila' 'five' 'left' 'happy'
 'bird' 'bird' 'yes' 'dog' 'bed' 'bed' 'three' 'bed' 'seven']
[False False  True False False False False False False False False False
 False False False False False False Fals